# GoogLeNet implementation

[Original video](https://youtu.be/uQc4Fs7yx5I)

[GoogLeNet paper](https://arxiv.org/abs/**1409.4842**)

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

In [2]:
# GoogLeNet architecture
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(conv_block, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))


class Inception_block(nn.Module):
    def __init__(self,
                 in_channels,
                 out_1x1,
                 red_3x3, out_3x3,
                 red_5x5, out_5x5,
                 out_1x1_pool):
        super(Inception_block, self).__init__()

        self.branch1 = conv_block(in_channels, out_1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            conv_block(in_channels, red_3x3, kernel_size=1),
            conv_block(red_3x3, out_3x3, kernel_size=3, padding=1)
        )

        # In Inception_v2 it has been shown to be more efficient
        # if instead 5x5 conv make two 3x3 convs.
        self.branch3 = nn.Sequential(
            conv_block(in_channels, red_5x5, kernel_size=1),
            conv_block(red_5x5, out_5x5, kernel_size=3, padding=1),
            conv_block(out_5x5, out_5x5, kernel_size=3, padding=1)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            conv_block(in_channels, out_1x1_pool, kernel_size=1)
        )

    def forward(self, x):
        # N x filters x 28 x 28
        return torch.cat([self.branch1(x),
                          self.branch2(x),
                          self.branch3(x),
                          self.branch4(x)], 1)


class GoogLeNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(GoogLeNet, self).__init__()

        self.conv1 = conv_block(in_channels=in_channels, out_channels=64,
                                kernel_size=(7,7), stride=(2,2), padding=(3,3))
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = conv_block(64, 192, kernel_size=3, stride=1, padding=1)

        # in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1_pool
        self.inception3a = Inception_block(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception_block(256, 128, 128, 192, 32, 96, 64)

        self.inception4a = Inception_block(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception_block(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception_block(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception_block(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception_block(528, 256, 160, 320, 32, 128, 128)

        self.inception5a = Inception_block(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception_block(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, 1000)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.pool(x)
        
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.pool(x)
        
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)

        x = x.reshape(x.shape[0], -1)  # flatten
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

In [3]:
x = torch.randn(64, 3, 224, 224)
model = GoogLeNet(in_channels=3, num_classes=1000)
print(model(x).shape)

torch.Size([64, 1000])


## Train the model

In [ ]:
# Hyperparameters
in_channels = 1
num_classes = 10
learning_rate = 1e-4
batch_size = 64
num_epochs = 4  # 40

load_model = True
filename = 'my_checkpoint.pth.tar'

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set transforms
my_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# For the error: HTTPError: HTTP Error 403: Forbidden
# StackOverflow: https://stackoverflow.com/a/66461122/7550928
from six.moves import urllib    
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

# Load data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=my_transforms, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=my_transforms, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize network
model = GoogLeNet(in_channels=in_channels, num_classes=num_classes).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)


def save_checkpoint(state, filename=filename):
    print(' => Saving checkpoint')
    torch.save(state, filename)


def load_checkpoint(checkpoint):
    print(' => Loading checkpoint')
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    best_acc = checkpoint['acc']


if load_model and os.path.exists(filename):
    load_checkpoint(torch.load(filename))
else:
    best_acc = 0


# Check accuracy on training and test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data', end='')
    else:
        print('Checking accuracy on test data', end='')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)  # shape = 64x10
            _, predictions = scores.max(dim=1)  # get index of max value
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        acc = float(num_correct) / float(num_samples) * 100
        #print(f'Got {num_correct} / {num_samples} with accuracy {acc:.2f}')

        model.train()
        return acc


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# Train network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to Cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)  # shape 64x10
        loss = criterion(scores, targets)
        losses.append(loss.item())

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()
    
    mean_loss = sum(losses)/len(losses)
    acc = check_accuracy(test_loader, model)
    msg = f'\rLoss at epoch {epoch} is {mean_loss:.5f}. Accuracy is {acc:.2f}'

    if best_acc < acc:
        best_acc = acc
        checkpoint = {'state_dict': model.state_dict(),
                      'optimizer': optimizer.state_dict(),
                      'acc': best_acc}
        print(msg, end='')
        save_checkpoint(checkpoint)
    else:
        print(msg)

Loss at epoch 0 is 0.02051. Accuracy is 99.16
Loss at epoch 1 is 0.01949. Accuracy is 99.46 => Saving checkpoint
Loss at epoch 2 is 0.01628. Accuracy is 99.51 => Saving checkpoint
Loss at epoch 3 is 0.01494. Accuracy is 99.19


In [ ]:
# MNIST accuracy for:
#   LeNet            = 97.38 %
#   VGG16 pretrained = 97.75 %
#   VGG16            = 98.73 %
#   GoogLeNet        = 99.51 %
print(f': {check_accuracy(train_loader, model):.2f}')
print(f': {check_accuracy(test_loader, model):.2f}')

Checking accuracy on training data: 99.34
Checking accuracy on test data: 99.19
